In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # library for regular expression operations
import string                              # for string operations

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer as ps  # module for stemming

# For building neural netwrok models
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import transformers

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time


In [2]:
# Load Data
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [3]:
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [4]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalence = binary_labels.aggregate('sum')/1237


# Combine the x_test and y_test sets
df = text_all.merge(binary_labels, on="id", how="left")

In [5]:
# Download the stopwords from NLTK
nltk.download('stopwords')

# Import the standard English stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
additional_sw = ['admission', 'date', 'discharge', 'summary', 'diagnoses', 'diagnosis', 'history',
                    'patient', 'report', 'status', 'signed', 'attending', 'service', 'dictated', 'dictation']
all_stopwords = additional_sw + stopwords_english

In [7]:
# # Convert all words in text column to lower case in both the train and test sets
# df['text'] = [str(x).lower() for x in df['text']]

# # Remove noiseb
# df['text']  = [re.sub("(\W|\d+|\n)", " ", elem).strip() for elem in df['text']] # remove spaces, digits and line breaks

# # Remove punctuations
# def remove_punctuations(text):
#     return ' '.join(['' if (elem in string.punctuation) else elem for elem in text.split()])

# df['text'] = [remove_punctuations(elem) for elem in df['text']]


# # Remove standard English stop words
# def remove_stopwords(text):
#     return ' '.join(['' if (elem in all_stopwords) else elem for elem in text.split()])

# df['text'] = [remove_stopwords(elem) for elem in df['text']]

# # Remove words with a length of one or two
# def remove_short_words(text):
#     return ' '.join(['' if (len(elem) <= 2) else elem for elem in text.split()])

# df['text'] = [remove_short_words(elem) for elem in df['text']]


In [8]:
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in all_stopwords) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    return(text)

In [9]:
df['text'] = [preprocessing(x) for x in df['text']]

In [10]:
# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
    pred = model.predict(X)
    pred = [1 if p>=0.5 else 0 for p in pred]
    acc = np.sum(y == pred)/len(y)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred)
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [11]:
# Create the CNN model
def cnnmodel(vocab_size, embedding_dim,max_length,dropout_rate):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(Conv1D(8, kernel_size=8, activation="relu"))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())  
    model.add(Dense(units=1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [12]:
# Create the RNN model
def rnnmodel(vocab_size, embedding_dim,max_length,dropout_rate):    
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)) # No pre-trained embedding
    model.add(layers.SimpleRNN(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [13]:
# Create the GRU model
def grumodel(vocab_size, embedding_dim,max_length,dropout_rate):    
    model = Sequential()   
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(layers.GRU(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [14]:
# Create the LSTM model
def lstmmodel(vocab_size, embedding_dim,max_length,dropout_rate):    
    model = Sequential() 
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(layers.LSTM(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [15]:
# Create the LSTM model
def bi_lstm_model(vocab_size, embedding_dim,max_length,dropout_rate):    
    model = Sequential() 
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
    model.add(layers.Bidirectional(layers.LSTM(8)))
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [17]:
# Set the number of epochs and the batch size
epochs = 20 
batch_size = 32
max_length = 557 # Make all sequences with length 557
embedding_dim = 200 # Dimension of word embedding
dropout_rate=0.3

In [18]:
# Run models
X = df['text']
models = [cnnmodel,rnnmodel,grumodel,lstmmodel,bi_lstm_model]
model_names = ['cnnmodel','rnnmodel','grumodel','lstmmodel','bi_lstm_model']


time_1 = time.time()

for i in range(16): # Go through each disease type 
    event_categorical = df['This_Disease_'+str(i)]
    dis_type = disease_types[i]
    
    time_start = time.time()   

    # Split the data 10 times into train and test sets with stratification
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25,random_state=0)
    
    j = 0
    for train_index, test_index in sss.split(X, event_categorical): 
               
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        tokenizer = Tokenizer(num_words=5000) # Tokenize the text
        tokenizer.fit_on_texts(x_train) 
        x_train = tokenizer.texts_to_sequences(x_train)
        x_test = tokenizer.texts_to_sequences(x_test)
        vocab_size = len(tokenizer.word_index) + 1 # plus the reserved index 0

        # Pad the sequences with 0's
        x_train = pad_sequences(x_train, padding='post', maxlen=max_length) 
        x_test = pad_sequences(x_test, padding='post', maxlen=max_length)

        # Fit the CNN model
        j += 1
        iteration = "iter" + str(j)
        
        for m in range(len(models)):    
            time_s = time.time()  
            mymodel = models[m]  
            model = mymodel(vocab_size, embedding_dim,max_length,dropout_rate)
            model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
            time_e = time.time() - time_s

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(model, x_test, y_test)

            with open('other/CNN_Paper_no_embedding_20epocs.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{model_names[m]},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{time_e},{vocab_size},{len(x_train)}\n')
    
    # Average metrics
    running_time = time.time() - time_start
    # Save model performance metrics
    with open('other/CNN_Paper_no_embedding_20epocs.csv','a') as fd:
        fd.write(f'{dis_type}, 10 iteration running time, {running_time}\n')

Epoch 1/20
29/29 [==============================] - 5s 149ms/step - loss: 0.4602 - auc: 0.5208
Epoch 2/20
29/29 [==============================] - 4s 144ms/step - loss: 0.3271 - auc: 0.9116
Epoch 3/20
29/29 [==============================] - 4s 153ms/step - loss: 0.2521 - auc: 0.9881
Epoch 4/20
29/29 [==============================] - 4s 151ms/step - loss: 0.1740 - auc: 0.9982
Epoch 5/20
29/29 [==============================] - 6s 198ms/step - loss: 0.0949 - auc: 1.0000
Epoch 6/20
29/29 [==============================] - 7s 244ms/step - loss: 0.0436 - auc: 1.0000
Epoch 7/20
29/29 [==============================] - 6s 212ms/step - loss: 0.0209 - auc: 1.0000
Epoch 8/20
29/29 [==============================] - 6s 213ms/step - loss: 0.0119 - auc: 1.0000
Epoch 9/20
29/29 [==============================] - 6s 193ms/step - loss: 0.0075 - auc: 1.0000
Epoch 10/20
29/29 [==============================] - 6s 194ms/step - loss: 0.0054 - auc: 1.0000 4s - los
Epoch 11/20
29/29 [=====================

29/29 [==============================] - 9s 304ms/step - loss: 0.2531 - auc: 0.8174
Epoch 12/20
29/29 [==============================] - 8s 285ms/step - loss: 0.2544 - auc: 0.8158
Epoch 13/20
29/29 [==============================] - 9s 308ms/step - loss: 0.2461 - auc: 0.8385
Epoch 14/20
29/29 [==============================] - 8s 287ms/step - loss: 0.2395 - auc: 0.8501
Epoch 15/20
29/29 [==============================] - 9s 300ms/step - loss: 0.2504 - auc: 0.8387
Epoch 16/20
29/29 [==============================] - 8s 288ms/step - loss: 0.2275 - auc: 0.8620
Epoch 17/20
29/29 [==============================] - 9s 301ms/step - loss: 0.2486 - auc: 0.8315
Epoch 18/20
29/29 [==============================] - 8s 287ms/step - loss: 0.2577 - auc: 0.8062
Epoch 19/20
29/29 [==============================] - 9s 299ms/step - loss: 0.2455 - auc: 0.8285
Epoch 20/20
29/29 [==============================] - 9s 293ms/step - loss: 0.2554 - auc: 0.8118
Epoch 1/20
29/29 [==============================] - 

29/29 [==============================] - 12s 311ms/step - loss: 0.5552 - auc: 0.4850
Epoch 2/20
29/29 [==============================] - 9s 325ms/step - loss: 0.4402 - auc: 0.5095
Epoch 3/20
29/29 [==============================] - 9s 317ms/step - loss: 0.4125 - auc: 0.5584
Epoch 4/20
29/29 [==============================] - 10s 334ms/step - loss: 0.4002 - auc: 0.5961
Epoch 5/20
29/29 [==============================] - 9s 322ms/step - loss: 0.3876 - auc: 0.6377
Epoch 6/20
29/29 [==============================] - 9s 319ms/step - loss: 0.3713 - auc: 0.6703
Epoch 7/20
29/29 [==============================] - 10s 337ms/step - loss: 0.3496 - auc: 0.7021
Epoch 8/20
29/29 [==============================] - 9s 312ms/step - loss: 0.3289 - auc: 0.7509
Epoch 9/20
29/29 [==============================] - 9s 327ms/step - loss: 0.3065 - auc: 0.7622
Epoch 10/20
29/29 [==============================] - 9s 312ms/step - loss: 0.2968 - auc: 0.7689
Epoch 11/20
29/29 [==============================] - 10s 

29/29 [==============================] - 6s 215ms/step - loss: 0.0054 - auc: 1.0000
Epoch 11/20
29/29 [==============================] - 5s 172ms/step - loss: 0.0038 - auc: 1.0000
Epoch 12/20
29/29 [==============================] - 5s 177ms/step - loss: 0.0028 - auc: 1.0000
Epoch 13/20
29/29 [==============================] - 5s 181ms/step - loss: 0.0024 - auc: 1.0000
Epoch 14/20
29/29 [==============================] - 5s 171ms/step - loss: 0.0019 - auc: 1.0000
Epoch 15/20
29/29 [==============================] - 5s 172ms/step - loss: 0.0016 - auc: 1.0000
Epoch 16/20
29/29 [==============================] - 5s 183ms/step - loss: 0.0014 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 5s 171ms/step - loss: 0.0012 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 5s 177ms/step - loss: 0.0010 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 5s 184ms/step - loss: 8.6792e-04 - auc: 1.0000
Epoch 20/20
29/29 [=============================

29/29 [==============================] - 9s 305ms/step - loss: 0.2550 - auc: 0.8185
Epoch 1/20
29/29 [==============================] - 16s 426ms/step - loss: 0.5433 - auc: 0.4949
Epoch 2/20
29/29 [==============================] - 12s 421ms/step - loss: 0.4041 - auc: 0.5922
Epoch 3/20
29/29 [==============================] - 12s 404ms/step - loss: 0.3483 - auc: 0.7845
Epoch 4/20
29/29 [==============================] - 12s 421ms/step - loss: 0.2935 - auc: 0.9008
Epoch 5/20
29/29 [==============================] - 12s 418ms/step - loss: 0.2407 - auc: 0.9409
Epoch 6/20
29/29 [==============================] - 12s 425ms/step - loss: 0.1765 - auc: 0.9795
Epoch 7/20
29/29 [==============================] - 12s 405ms/step - loss: 0.1350 - auc: 0.9830
Epoch 8/20
29/29 [==============================] - 12s 419ms/step - loss: 0.1058 - auc: 0.9870
Epoch 9/20
29/29 [==============================] - 12s 419ms/step - loss: 0.0943 - auc: 0.9841
Epoch 10/20
29/29 [==============================] -

...

29/29 [==============================] - 5s 185ms/step - loss: 0.0025 - auc: 1.0000
Epoch 16/20
29/29 [==============================] - 5s 183ms/step - loss: 0.0021 - auc: 1.0000
Epoch 17/20
29/29 [==============================] - 5s 179ms/step - loss: 0.0019 - auc: 1.0000
Epoch 18/20
29/29 [==============================] - 5s 183ms/step - loss: 0.0015 - auc: 1.0000
Epoch 19/20
29/29 [==============================] - 5s 188ms/step - loss: 0.0014 - auc: 1.0000
Epoch 20/20
29/29 [==============================] - 5s 180ms/step - loss: 0.0013 - auc: 1.0000
Epoch 1/20
29/29 [==============================] - 7s 215ms/step - loss: 0.6568 - auc: 0.5351
Epoch 2/20
29/29 [==============================] - 6s 203ms/step - loss: 0.5333 - auc: 0.7520
Epoch 3/20
29/29 [==============================] - 6s 207ms/step - loss: 0.4752 - auc: 0.8062
Epoch 4/20
29/29 [==============================] - 6s 216ms/step - loss: 0.4442 - auc: 0.8493
Epoch 5/20
29/29 [==============================] - 6s 2

29/29 [==============================] - 12s 428ms/step - loss: 0.3119 - auc: 0.9494
Epoch 6/20
29/29 [==============================] - 12s 428ms/step - loss: 0.2201 - auc: 0.9819
Epoch 7/20
29/29 [==============================] - 12s 431ms/step - loss: 0.1608 - auc: 0.9898
Epoch 8/20
29/29 [==============================] - 12s 414ms/step - loss: 0.1353 - auc: 0.9886
Epoch 9/20
29/29 [==============================] - 12s 426ms/step - loss: 0.1051 - auc: 0.9948
Epoch 10/20
29/29 [==============================] - 12s 427ms/step - loss: 0.0712 - auc: 0.9995
Epoch 11/20
29/29 [==============================] - 12s 426ms/step - loss: 0.0590 - auc: 0.9981
Epoch 12/20
29/29 [==============================] - 12s 415ms/step - loss: 0.0575 - auc: 0.9970
Epoch 13/20
29/29 [==============================] - 12s 425ms/step - loss: 0.0786 - auc: 0.9918
Epoch 14/20
29/29 [==============================] - 12s 429ms/step - loss: 0.0600 - auc: 0.9973
Epoch 15/20
29/29 [===========================

29/29 [==============================] - 10s 331ms/step - loss: 0.4084 - auc: 0.7996
Epoch 16/20
29/29 [==============================] - 9s 317ms/step - loss: 0.4027 - auc: 0.7980
Epoch 17/20
29/29 [==============================] - 10s 329ms/step - loss: 0.3970 - auc: 0.7965
Epoch 18/20
29/29 [==============================] - 9s 324ms/step - loss: 0.3920 - auc: 0.8044
Epoch 19/20
29/29 [==============================] - 9s 324ms/step - loss: 0.3883 - auc: 0.8149
Epoch 20/20
29/29 [==============================] - 10s 330ms/step - loss: 0.3925 - auc: 0.8117
Epoch 1/20
29/29 [==============================] - 11s 290ms/step - loss: 0.6365 - auc: 0.5074
Epoch 2/20
29/29 [==============================] - 9s 302ms/step - loss: 0.5923 - auc: 0.5276
Epoch 3/20
29/29 [==============================] - 9s 306ms/step - loss: 0.5760 - auc: 0.5711
Epoch 4/20
29/29 [==============================] - 9s 293ms/step - loss: 0.5599 - auc: 0.6137
Epoch 5/20
29/29 [==============================] - 

29/29 [==============================] - 6s 206ms/step - loss: 0.4854 - auc: 0.7770
Epoch 6/20
29/29 [==============================] - 6s 207ms/step - loss: 0.4608 - auc: 0.8364
Epoch 7/20
29/29 [==============================] - 6s 219ms/step - loss: 0.4443 - auc: 0.8513
Epoch 8/20
29/29 [==============================] - 6s 208ms/step - loss: 0.4099 - auc: 0.8934
Epoch 9/20
29/29 [==============================] - 6s 218ms/step - loss: 0.3723 - auc: 0.9228
Epoch 10/20
29/29 [==============================] - 6s 215ms/step - loss: 0.3431 - auc: 0.9400
Epoch 11/20
29/29 [==============================] - 6s 204ms/step - loss: 0.3067 - auc: 0.9527
Epoch 12/20
29/29 [==============================] - 6s 223ms/step - loss: 0.2793 - auc: 0.9659
Epoch 13/20
29/29 [==============================] - 6s 210ms/step - loss: 0.2679 - auc: 0.9627
Epoch 14/20
29/29 [==============================] - 6s 222ms/step - loss: 0.2389 - auc: 0.9744
Epoch 15/20
29/29 [==============================] - 6s 

29/29 [==============================] - 13s 443ms/step - loss: 0.0553 - auc: 0.9978
Epoch 16/20
29/29 [==============================] - 12s 430ms/step - loss: 0.0556 - auc: 0.9972
Epoch 17/20
29/29 [==============================] - 13s 439ms/step - loss: 0.0490 - auc: 0.9980
Epoch 18/20
29/29 [==============================] - 12s 429ms/step - loss: 0.0465 - auc: 0.9988
Epoch 19/20
29/29 [==============================] - 13s 438ms/step - loss: 0.0418 - auc: 0.9996
Epoch 20/20
29/29 [==============================] - 13s 436ms/step - loss: 0.0340 - auc: 0.9998
Epoch 1/20
29/29 [==============================] - 6s 186ms/step - loss: 0.6015 - auc: 0.5010 3s - loss: 0
Epoch 2/20
29/29 [==============================] - 5s 189ms/step - loss: 0.4832 - auc: 0.8962 4s - l
Epoch 3/20
29/29 [==============================] - 5s 177ms/step - loss: 0.3792 - auc: 0.9915
Epoch 4/20
29/29 [==============================] - 6s 192ms/step - loss: 0.2474 - auc: 0.9999
Epoch 5/20
29/29 [============

In [19]:
time_2 = time.time() - time_1  

In [20]:
time_2
with open('other/CNN_Paper_no_embedding_20epocs.csv','a') as fd:
    fd.write(f'{dis_type}, Total running time, {time_2}\n')